In [ ]:
!pip install -q langchain faiss-cpu google-cloud-aiplatform vertexai PyPDF2

In [ ]:
from google.colab import auth
auth.authenticate_user()

PROJECT_ID = "your-project-id"
REGION = "us-central1"
!gcloud config set project $PROJECT_ID

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

loader = PyPDFLoader("your_docs/product_guide.pdf")
docs = loader.load()
splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import VertexAIEmbeddings

embedding_model = VertexAIEmbeddings()
db = FAISS.from_documents(chunks, embedding_model)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatVertexAI

llm = ChatVertexAI()
qa = RetrievalQA.from_chain_type(llm=llm, retriever=db.as_retriever())

In [ ]:
question = "What is the refund policy?"
response = qa.run(question)
print("Q:", question)
print("A:", response)

In [ ]:
import json
from google.colab import files

uploaded = files.upload()
with open("golden_set.json") as f:
    golden_data = json.load(f)

In [ ]:
for i, item in enumerate(golden_data):
    q = item['question']
    expected = item['answer']
    actual = qa.run(q)
    print(f"\n[{i+1}] Q: {q}\nExpected: {expected}\nGenerated: {actual}")